In [ ]:
%pip install sklearn
%pip transformers
%pip install huggingface
%pip install torch 
%pip install pandas
%pip install sklearn
%pip install datasets
%pip install transformers

In [2]:
def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions = predictions, references = label)

In [3]:
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

from spacy import tokenizer
from spacy.lang.en import English
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer,AdamW,get_scheduler

In [ ]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher", "/data") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

In [62]:
#change to bypublisher because byarticle has no test set.
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = 32)
hyperpartisan_valid = hyperpartisan_valid.shuffle(seed = 32)

In [69]:
eval_test_split = hyperpartisan_valid.train_test_split(test_size = 0.5)

hyperpartisan_eval = eval_test_split['train']
hyperpartisan_test = eval_test_split['test']

In [45]:
hyperpartisan_eval

Dataset({
    features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
    num_rows: 480000
})

In [70]:
int_labels_train = [1 if x == True else 0 for x in hyperpartisan_train['hyperpartisan']]
int_labels_eval = [1 if x == True else 0 for x in hyperpartisan_eval['hyperpartisan']]
int_labels_test = [1 if x == True else 0 for x in hyperpartisan_test['hyperpartisan']]

In [ ]:
from transformers import LongformerTokenizer, LongformerForSequenceClassification,LongformerConfig

longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

longformer_model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,
                                                  num_labels = 2,
                                                  cache_dir='/models/longformer',
                                                                     return_dict=True)


WITHOUT FINETUNING

In [ ]:
with torch.no_grad():
  input = longformer_tokenizer(hyperpartisan_train['text'], return_tensors= "pt")
  out = longformer_model(**input).logits

  labels = out.argmax().item()

In [ ]:
labels, hyperpartisan_train['hyperpartisan']

TOKENIZING

In [67]:
def tokenize_num_of_files(files,labels, num = 20):
  """Select number of files to tokenize, outputs dictionary of attention mask,input ids and corresponding label"""
  if isinstance(files,list):
    input = longformer_tokenizer(files[:num])
    input["labels"] = labels[:num]
  else:
    input = longformer_tokenizer(files)
    if len(labels) == 1:
      input['labels'] = labels
    else:
      print("input single label for single text file")
  return input

In [ ]:
training_input = tokenize_num_of_files(hyperpartisan_train['text'], int_labels_train)

In [72]:
eval = tokenize_num_of_files(hyperpartisan_eval['text'], int_labels_eval, len(int_labels_eval))

Token indices sequence length is longer than the specified maximum sequence length for this model (5371 > 4096). Running this sequence through the model will result in indexing errors


In [73]:
eval

FINETUNING

In [ ]:
%pip install optuna

In [28]:
import optuna

In [41]:
def hp_space(trial):
  return {"per_device_train_batch_size":trial.suggest_int("per_device_train_batch_size", 8,16,8),
          "num_train_epochs":trial.suggest_int("num_training_epochs",1,5),
          "learning_rate": trial.suggest_float("learning_rate",1e-5,2e-5),
  }

In [51]:
config = LongformerConfig()

def longformer_init():
  return (LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,
                                                  num_labels = 2,
                                                  cache_dir='/models/longformer',
                                                                     return_dict=True))

data_collator = DataCollatorWithPadding(tokenizer=longformer_tokenizer) #to pad sequences

#default training arguments 
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=16, 
                                  evaluation_strategy= "epoch", per_device_eval_batch_size=16, num_train_epochs=5)

#fine-tuning model
longformer_finetuned = Trainer(model_init = longformer_init, 
              model = longformer_model,
              args = training_args,
              train_dataset= hyperpartisan_train,
              eval_dataset= hyperpartisan_eval,
              tokenizer= longformer_tokenizer,
              data_collator=data_collator,
              compute_metrics= compute_metrics)

#searching for best hyperparameters
best_run = longformer_finetuned.hyperparameter_search(direction = "maximize", hp_space = hp_space)





PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:341: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  FutureWarning,
[I 2022-07-27 06:55:35,828] A new study created in memory with name: no-name-bbb28ea5-a99c-479f-a118-9dd0469f85f5
Trial:
loading configuration file https://huggingface.co/allenai/longformer-base-4096/resolve/main/config.json from cache at /models/longformer/0690955d8f70934f95adf0fb108d5f7322d02f8d7dd938b7b133cb7421e120e6.b25f41ff6acdcb7ab47c505c70e351b3fc01957b3798197e5ac6e8efc547ac99
Model config Lon

ValueError: ignored

In [ ]:
#plot 
fig, ax = optuna.visualization.plot_slice(study, params=["x", "y"])

In [ ]:
#update hyperparameters 
for k,v in best_run.hyperparameters.items():
  setattr(training_args, k, v)

best_trainer = Trainer(model = longformer_model,
                       args = training_args,
                       train_dataset= hyperpartisan_train,
                      eval_dataset= hyperpartisan_eval,
                      tokenizer= longformer_tokenizer,
                      data_collator=data_collator,
                      compute_metrics= compute_metrics)

best_trainer.train()